In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2
from binaryIV import simulate_deterministic_data_with_probabilistic_ate, extract_prob_dict, entropy_of_array

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



import matplotlib.pyplot as plt
import seaborn as sns
import math
# from linearmodels.iv import IV2SLS
import statsmodels.api as sm
import statsmodels.formula.api as smf


pd.options.mode.chained_assignment = None  # default='warn'


pd.set_option('display.max_columns', None)




In [ ]:
# Framework to process each line of the CSV (excluding the header)
def process_csv_line_by_line(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        # Read and write the header
        header = infile.readline()
        outfile.write(header)

        # Process each line
        for line in infile:
            # Replace specific commas with decimal points
            parts = line.strip().split(',')
            b_X_Y_1000 = parts[0]
            # Merge 2nd and 3rd part, with a point in between
            zaffalon_bound_lower = parts[1] + '.' + parts[2]
            # Merge 4th and 5th part, with a point in between
            zaffalon_bound_upper = parts[3] + '.' + parts[4]
            outfile.write(b_X_Y_1000 + ',' + zaffalon_bound_lower + ',' + zaffalon_bound_upper + '\n')

# Apply the framework to fix the CSV
process_csv_line_by_line('results_parallel.csv', 'results_parallel_fixed.csv')

# Load the fixed CSV into a DataFrame
zaffalon_results = pd.read_csv('results_parallel_fixed.csv')
zaffalon_results

['-4965', '-0', '4503', '-0', '3038']
['-4970', '-0', '3556', '0', '4701']
['-4990', '-0', '5913', '-0', '3255']
['-4985', '-0', '5807', '-0', '0126']
['-4975', '-0', '7200', '-0', '1801']
['-4980', '-0', '8183', '-0', '0336']
['-5000', '-0', '7038', '-0', '2233']
['-4995', '-0', '5703', '-0', '3486']
['-4955', '-0', '5848', '-0', '4258']
['-4960', '-0', '6898', '-0', '3293']
['-4935', '-0', '7525', '0', '1584']
['-4945', '-0', '5853', '0', '1699']
['-4950', '-0', '6432', '-0', '3409']
['-4940', '-0', '5617', '-0', '3862']
['-4930', '-0', '5065', '-0', '1254']
['-4925', '-0', '6599', '0', '0408']
['-4910', '-0', '5202', '0', '1403']
['-4905', '-0', '5287', '-0', '3597']
['-4915', '-0', '4884', '0', '0517']
['-4900', '-0', '3098', '0', '2398']
['-4895', '-0', '5358', '0', '2849']
['-4920', '-0', '7067', '0', '0113']
['-4890', '-0', '8199', '0', '0426']
['-4885', '-0', '4295', '0', '2172']
['-4880', '-0', '6346', '-0', '1250']
['-4845', '-0', '5338', '0', '2304']
['-4875', '-0', '6183', 

,b_X_Y_1000,zaffalon_bound_lower,zaffalon_bound_upper
0,-4965,-0.4503,-0.3038
1,-4970,-0.3556,0.4701
2,-4990,-0.5913,-0.3255
3,-4985,-0.5807,-0.0126
4,-4975,-0.7200,-0.1801
...,...,...,...
1995,4970,0.0858,0.5280
1996,4975,-0.1414,0.4909
1997,4985,0.3481,0.7105
1998,4990,-0.0840,0.7075
